<a href="https://colab.research.google.com/github/manuel-alvarez/scheduling/blob/master/poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schedule Generator with Genetic Algorithms

This POC is intended to be a schedule generator for schools.

There are three type of resources:
 - Classrooms
 - Teachers
 - Subjects

For this POC we will consider that all students in a class go toguether and don't move, so they are something like attached to the classroom and therefore not considered as resources, but they could.

## POC

In this first POC, we are going to consider two of the three types of resources as constant, and one of them as variable. For example:
 - 1 classroom
 - 1 teacher
 - Multiple subjects

This way, we just need to use resources of one type.


In [0]:
import numpy

In [0]:
# Gene population
POPULATION_SIZE = 20
# Each segment represents a day
SEGMENT = 5
# Five days, since the schedule is weekly based, that are 5 segments
INDIVIDUAL_SIZE = 25
# Resources, currently, of just one type, let's say subjects
RESOURCES = 5
# Number of genes that pass to the next generation
SURVIVAL_RATE = .4
# Rate of elements that are goint to mutate
MUTATION_RATE = .05
# Number of iterations we do in order to get the best approach
STEPS = 100

In [0]:
def initialize_population(population_size, individual_size, resources_size):
  population = None
  for i in range(population_size):
    if population is None:
      population = numpy.array([numpy.random.randint(resources_size + 1, size=individual_size)])
    else:
      population = numpy.append(population, [numpy.random.randint(resources_size + 1, size=individual_size)], axis=0)
  return population

In [0]:
def get_fitness(the_try):
  fitness = 0
  for i in range(int(numpy.ceil(INDIVIDUAL_SIZE / SEGMENT))):
    # print(f'Evaluating', i*SEGMENT, (i+1)*SEGMENT, the_try[i*SEGMENT:(i+1)*SEGMENT], 'from', the_try)
    unique, counts = numpy.unique(the_try[i*SEGMENT:(i+1)*SEGMENT], return_counts=True)
    fitness += sum(item for item in counts if item > 1)  # Not repeated resources in same segment
    fitness += 1 if 0 in unique else 0  # Not empty slots in segment
  return fitness

In [0]:
def get_selection(population):
  return numpy.array(sorted([[get_fitness(the_try), the_try] for the_try in population], key=lambda x:x[0])[:int(numpy.ceil(POPULATION_SIZE * SURVIVAL_RATE))])

In [0]:
def breed(selection):
  population = numpy.array([the_try for the_try in selection])
  i = 0
  while len(population) < POPULATION_SIZE:
    parents = population[i:i+2]
    numpy.random.shuffle(parents)
    slicery = numpy.random.randint(INDIVIDUAL_SIZE) 
    population = numpy.append(population, numpy.array([numpy.append(parents[0][slicery:], parents[1][:slicery], axis=0)]), axis=0)
    i += 1
    if i == len(selection):
      i = 0
  return population

In [0]:
def mutate(population):
  mutants = int(numpy.ceil(POPULATION_SIZE * MUTATION_RATE))
  for i in range(mutants):
    index = numpy.random.randint(POPULATION_SIZE)
    mutations = numpy.random.randint(RESOURCES, size=4)
    mutant = population[index]
    for mutation in mutations:
      mutant[numpy.random.randint(INDIVIDUAL_SIZE)] = mutation
    population[index] = mutant
  return population

In [40]:
population = initialize_population(POPULATION_SIZE, INDIVIDUAL_SIZE, RESOURCES)
for i in range(STEPS):
  print(f'Step {i}')
  selection = get_selection(population)
  print(selection[0:1])
  population = breed(selection[:,1])
  population = mutate(population)
print(numpy.reshape(population[0], (5, 5)))

Step 0
[[8
  array([5, 1, 4, 3, 0, 1, 5, 2, 4, 0, 1, 5, 4, 1, 2, 2, 3, 1, 2, 0, 2, 3,
       0, 1, 4])]]
Step 1
[[8
  array([5, 1, 4, 3, 0, 1, 5, 2, 4, 0, 1, 5, 4, 1, 2, 2, 3, 1, 2, 0, 2, 3,
       0, 1, 4])]]
Step 2
[[6
  array([0, 4, 1, 2, 2, 3, 1, 5, 0, 4, 0, 1, 3, 4, 5, 1, 4, 3, 0, 2, 5, 2,
       4, 3, 1])]]
Step 3
[[6
  array([0, 4, 1, 2, 2, 3, 1, 5, 0, 4, 0, 1, 3, 4, 5, 1, 4, 3, 0, 2, 5, 2,
       4, 3, 1])]]
Step 4
[[6
  array([0, 4, 1, 2, 2, 3, 1, 5, 0, 4, 0, 1, 3, 4, 5, 1, 4, 3, 0, 2, 5, 2,
       4, 3, 1])]]
Step 5
[[5
  array([2, 0, 3, 5, 1, 4, 3, 0, 1, 5, 2, 4, 0, 1, 5, 5, 1, 4, 3, 0, 1, 5,
       2, 4, 0])]]
Step 6
[[4
  array([0, 1, 3, 4, 5, 1, 4, 3, 0, 2, 5, 2, 4, 3, 1, 2, 0, 3, 5, 1, 4, 3,
       0, 1, 5])]]
Step 7
[[4
  array([0, 1, 3, 4, 5, 1, 4, 3, 0, 2, 5, 2, 4, 3, 1, 2, 0, 3, 5, 1, 4, 3,
       0, 1, 5])]]
Step 8
[[4
  array([0, 1, 3, 4, 5, 1, 4, 3, 0, 2, 5, 2, 4, 3, 1, 2, 0, 3, 5, 1, 4, 3,
       0, 1, 5])]]
Step 9
[[4
  array([0, 1, 3, 4, 5, 1, 4, 3, 0, 2, 5, 2,